In [77]:
import numpy as np
import math
from prettytable import PrettyTable

In [75]:
data = """0.698132 0.188224
0.959931 0.209138
1.134464 0.230052
1.570796 0.250965
1.919862 0.313707"""
d2 = """0 1 3 5 7 9
1.000 0.891 0.708 0.562 0.447 0.355"""

data = """ """

In [105]:
def data_extract(data):
    data = data.replace('×10','e')
    data = data.replace('−','-')

    data = data.split("\n")
    dim = len(data)
    for i in range(dim):
        data[i] = data[i].split(' ')
        data[i] = [float(x) for x in data[i]]
    
    data = np.array(data)
    data = data.T if data.shape[0]<data.shape[1] else data
    return data

def lin_reg_data(data):
    data = data_extract(data)
    # x = np.reshape(data[:,0],(data[:,0].shape[0],1))
    # y = np.reshape(data[:,1],(data[:,1].shape[0],1))
    x = np.array([data[:,0]]).T
    y = np.array([data[:,1]]).T
    ###################################################################
    x = x
    #x = np.log(x)
    #x = np.pow(x,-1)
    
    #y = y
    y = np.log(y)
    #y = np.pow(y,-1)
    ###################################################################
    data = np.append(data,x,axis=1)
    data = np.append(data,y,axis=1)
    data = np.append(data,x**2,axis=1)
    data = np.append(data,x*y,axis=1)
    
    tab = PrettyTable()
    tab.field_names = ["X'","Y'",'X','Y','X^2','XY']
    [tab.add_row(data[i,:]) for i in range(data.shape[0])]

    tab.add_row(['-'*7]*6)
    tab.add_row([f"Σ = {np.sum(data[:,i])}" for i in range(data.shape[1])])

    print(tab)

    X = np.ones(x.shape)
    X = [np.append(X,x**i,axis = 1) for i in range(1,2)][0]

    SS = np.linalg.inv(X.T@X)@X.T@y
    print("Unprocessed A B\n",SS)


    print("X is",X)

    #######################
    #A = SS[0]
    A = math.exp(SS[0])
    #A = 1/SS[0,0]

    B = SS[1,0]
    #B = B/A
    print("Final ans a0 a1",A,B)
    #######################

lin_reg_data(d2)


+----------+------------------------+----------+-------------------------+-----------+------------------------+
|    X'    |           Y'           |    X     |            Y            |    X^2    |           XY           |
+----------+------------------------+----------+-------------------------+-----------+------------------------+
|   0.0    |          1.0           |   0.0    |           0.0           |    0.0    |          0.0           |
|   1.0    |         0.891          |   1.0    |   -0.11541085151132773  |    1.0    |  -0.11541085151132773  |
|   3.0    |         0.708          |   3.0    |   -0.34531118528841737  |    9.0    |   -1.035933555865252   |
|   5.0    |         0.562          |   5.0    |   -0.5762534290884459   |    25.0   |  -2.8812671454422296   |
|   7.0    |         0.447          |   7.0    |   -0.8051966843685682   |    49.0   |   -5.636376790579977   |
|   9.0    |         0.355          |   9.0    |   -1.0356374895067213   |    81.0   |   -9.320737405560

In [101]:
def polly_reg(data,order=2):
    data = data_extract(data)
    # x = np.reshape(data[:,0],(data[:,0].shape[0],1))
    # y = np.reshape(data[:,1],(data[:,1].shape[0],1))
    x = np.array([data[:,0]]).T
    y = np.array([data[:,1]]).T

    data = np.append(data,x**2,axis=1)
    data = np.append(data,x**3,axis=1)
    data = np.append(data,x**4,axis=1)

    data = np.append(data,y*x,axis=1)
    data = np.append(data,y*x**2,axis=1)

    data = np.around(data,decimals=10)

    tab = PrettyTable()
    tab.field_names = ["X","Y",'X^2','X^3','X^4','YX','YX^2']
    [tab.add_row(data[i,:]) for i in range(data.shape[0])]

    tab.add_row(['-'*7]*7)
    tab.add_row([f"Σ = {np.sum(data[:,i])}" for i in range(data.shape[1])])

    print(tab)

    n = len(x)
    M = np.zeros((order+1, order+1))
    M[0][0] = n
    b = np.zeros((order+1, 1))

    for i in range(order+1):
        if(i == 0):
            b[i][0] = np.sum(y)
        else:
            b[i][0] = np.sum(np.power(x, i) * y)


    for i in range(order+1):
        for j in range(order+1):
            if(i+j == 0):
                M[i][j] = n
            else:
                M[i][j] = np.sum(np.power(x, i+j))
    
    A = np.dot(np.linalg.inv(M), b)
    print(A)




In [102]:
data = """80 6.47×10−6
40 6.24×10−6
-40 5.72×10−6
-120 5.09×10−6
-200 4.30×10−6
-280 3.33×10−6
-340 2.45×10−6"""

polly_reg(data)

+------------+--------------+--------------+-----------------+-------------------+----------------+------------------------+
|     X      |      Y       |     X^2      |       X^3       |        X^4        |       YX       |          YX^2          |
+------------+--------------+--------------+-----------------+-------------------+----------------+------------------------+
|    80.0    |   6.47e-06   |    6400.0    |     512000.0    |     40960000.0    |   0.0005176    |        0.041408        |
|    40.0    |   6.24e-06   |    1600.0    |     64000.0     |     2560000.0     |   0.0002496    |        0.009984        |
|   -40.0    |   5.72e-06   |    1600.0    |     -64000.0    |     2560000.0     |   -0.0002288   |        0.009152        |
|   -120.0   |   5.09e-06   |   14400.0    |    -1728000.0   |    207360000.0    |   -0.0006108   |        0.073296        |
|   -200.0   |   4.3e-06    |   40000.0    |    -8000000.0   |    1600000000.0   |    -0.00086    |         0.172          |
